In [15]:
import os
import pandas as pd
import pretty_midi
from music21 import converter
import numpy as np

In [ ]:


base_dir = './dataset/'  # Ajusta según tu ruta
dataset_info = []         # Lista donde guardaremos toda la información

for artist in os.listdir(base_dir):
    artist_path = os.path.join(base_dir, artist)
    if os.path.isdir(artist_path):
        for file_name in os.listdir(artist_path):
            if file_name.lower().endswith(('.mid', '.midi')):
                midi_path = os.path.join(artist_path, file_name)
                
                # ============ 1) Parsear con music21 para metadatos ============
                score = converter.parse(midi_path)
                
                # Obtener el compositor (si está en los metadatos)
                composer = None
                if score.metadata is not None:
                    composer = score.metadata.composer
                
                # (Opcional) Analizar la tonalidad
                try:
                    key_obj = score.analyze('key')
                    key_str = f"{key_obj.tonic.name} {key_obj.mode}"
                except:
                    key_str = None
                
                # ============ 2) Convertir a Piano Roll con pretty_midi ============
                pm = pretty_midi.PrettyMIDI(midi_path)
                # Obtenemos un piano roll combinado de todos los instrumentos
                # Dimensiones: (128, time_steps), donde 128 son las notas MIDI
                piano_roll = pm.get_piano_roll(fs=100)  
                # fs=100 => frames por segundo (puedes ajustarlo)

                # (Opcional) Binarizar si quieres un piano roll "discreto"
                # piano_roll = (piano_roll > 0).astype(int)

                # ============ 3) Guardar en nuestra lista de datos ============
                dataset_info.append({
                    'artist': artist,         # Nombre de la carpeta
                    'composer': composer,     # Metadato extraído de music21
                    'key': key_str,           # Tonalidad (si se pudo extraer)
                    'midi_path': midi_path,   # Ruta del archivo MIDI
                    'piano_roll': piano_roll  # Matriz numpy con la partitura
                })

# Convertimos la lista a DataFrame
df = pd.DataFrame(dataset_info)

# Mostrar un resumen
print(df.head())


/home/james/.local/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2005 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
/home/james/.local/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 1997 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
/home/james/.local/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=5, channel=None, data=b'Copyright \xa9 2006 by Bernd Kr\xfcger'>; getting generic Instrument
  warnings.warn(
/home/james/.local/lib/python3.10/site-packages/music21/midi/translate.py:874: TranslateWarning: Unable to determine i

   artist composer       key                      midi_path  \
0  mozart     None   F major  ./dataset/mozart/mz_332_1.mid   
1  mozart     None  B- major  ./dataset/mozart/mz_333_1.mid   
2  mozart     None  E- major  ./dataset/mozart/mz_333_2.mid   
3  mozart     None   D major  ./dataset/mozart/mz_311_1.mid   
4  mozart     None  B- major  ./dataset/mozart/mz_332_2.mid   

                                          piano_roll  
0  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
1  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
2  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
3  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  
4  [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...  


In [22]:
# Save DataFrame to a CSV file
df.to_csv('data.csv', index=False)

In [37]:
y = [np.unique(i) for i in df["piano_roll"]]

In [38]:
y

[array([ 0., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30., 31., 32.,
        33., 34., 35., 36., 37., 38., 39., 40., 41., 42., 43., 44., 45.,
        46., 47., 48., 49., 50., 51., 52., 53., 54., 55., 56., 57., 58.,
        59., 60., 62., 63., 64., 65., 66., 67., 68., 69., 70., 71., 72.,
        73., 74., 75., 76., 78., 79., 80., 81., 82., 84., 85., 86., 87.,
        88., 89., 90., 91.]),
 array([ 0., 24., 27., 28., 29., 30., 31., 32., 33., 34., 35., 36., 37.,
        38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48., 49., 50.,
        51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61., 62., 63.,
        64., 65., 66., 67., 68., 69., 70., 71., 72., 73., 74., 75., 76.,
        77., 78., 79., 80., 81., 82., 83., 84., 85., 87., 88., 89., 90.,
        91., 92., 93., 94., 95., 96., 97.]),
 array([ 0., 23., 24., 25., 26., 27., 29., 30., 31., 32., 33., 34., 35.,
        36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48.,
        49., 50., 51., 52., 53., 54., 55., 56., 5